In [37]:
import sys
sys.path.insert(0, '../')

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.dummy import DummyClassifier

from datasets.mqtt_dataset import MqttDataset

In [55]:
train_dataset = MqttDataset("../raw_data/mqtt_fullrows.csv", is_train=True, include_malicious_traffic=False, lbl_is_src_ip=True)
test_dataset = MqttDataset("../raw_data/mqtt_fullrows.csv", is_train=False, include_malicious_traffic=False, lbl_is_src_ip=True)

In [56]:
X_train, y_train = train_dataset.X, train_dataset.y
X_test, y_test = test_dataset.X, test_dataset.y

In [54]:
def anlyse(X_train, y_train, X_test, y_test, importanceTH=0.1):
    clf = RandomForestClassifier(n_jobs=80, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'accuracy_score :{accuracy_score(y_test, y_pred)}')

    print('feature importance :', sorted(clf.feature_importances_.tolist(), reverse=True)[:5])
    print('importent features :', np.where(clf.feature_importances_ > 0.1)[0])

    dummy(X_train, y_train, X_test, y_test)

    for col in np.where(clf.feature_importances_ > 0.1)[0]:
        print(train_dataset.clms[col])


def dummy(X_train, y_train, X_test, y_test):
    dummy = DummyClassifier()
    dummy.fit(X_train, y_train)
    y_pred = dummy.predict(X_test)
    print(f'dummy accurcy : {accuracy_score(y_test, y_pred)}')




def dropColums(X_train, X_test, colums):
    for col in colums:
        X_train = np.hstack([X_train[:, :col], X_train[:, (col+1):]])
        X_test = np.hstack([X_test[:, :col], X_test[:, (col+1):]])

    return X_train, X_test

        

In [57]:
anlyse(X_train, y_train, X_test, y_test)

accuracy_score :0.022710155842651063
feature importance : [0.9796852061621744, 0.010528674091017405, 0.009786119746808182, 0.0, 0.0]
importent features : [8]
dummy accurcy : 0.08049939169225422
tcp_flag_res


In [58]:
X_train, X_test = dropColums(X_train, X_test, [8])

In [59]:
anlyse(X_train, y_train, X_test, y_test)

accuracy_score :0.09579398644342738
feature importance : [0.5472798139058487, 0.4527201860941513, 0.0, 0.0, 0.0]
importent features : [ 5 19]
dummy accurcy : 0.08049939169225422
ip_flag_rb
mqtt_messagelength


In [60]:
X_train, X_test = dropColums(X_train, X_test, [5])

In [61]:
anlyse(X_train, y_train, X_test, y_test)

accuracy_score :0.09579398644342738
feature importance : [1.0, 0.0, 0.0, 0.0, 0.0]
importent features : [18]
dummy accurcy : 0.08049939169225422
mqtt_messagetype


In [62]:
X_train, X_test = dropColums(X_train, X_test, [18])

In [64]:
anlyse(X_train, y_train, X_test, y_test)

accuracy_score :0.08049939169225422
feature importance : [0.0, 0.0, 0.0, 0.0, 0.0]
importent features : []
dummy accurcy : 0.08049939169225422
